## 타이타닉 데이터 다운로드

In [3]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"taeyangg4","key":"b9a22b3cb91a965bc5ca1c46065ee041"}'}

In [5]:
# ls -1ha kaggle.json

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600  ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 19.7MB/s]


In [10]:
# !ls
!unzip /content/titanic.zip

Archive:  /content/titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# import & sklearn 

In [21]:
# !pip list
!pip install xgboost
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 4.5 MB/s eta 0:00:00


In [335]:
# 데이터 관리
import pandas as pd
import numpy as np
import pickle
import datetime
import pytz
import time
import math
import os

# 시각화
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
from scipy.stats import norm

# 모델링
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from category_encoders import OneHotEncoder, OrdinalEncoder, TargetEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc

In [814]:
data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')
submission = pd.read_csv('/content/gender_submission.csv')

# EDA 및 전처리

In [815]:
test = test_data.copy()

In [816]:
# df = pd.concat([data,test_data],axis = 0).reset_index(drop = True)
df = data.copy()
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [636]:
# # 1st EDA
# df = data.copy()
# df['Fare'] = pd.qcut(df['Fare'], 13)
# df['Age'] = pd.qcut(df['Age'], 10)
# df['Family_Size'] = df['SibSp'] + df['Parch'] + 1
# df['Ticket_Frequency'] = df.groupby('Ticket')['Ticket'].transform('count')

# df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
# df['Is_Married'] = 0
# df['Is_Married'].loc[df['Title'] == 'Mrs'] = 1

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [826]:

# 2nd EDA
#Thanks to:
# https://www.kaggle.com/mauricef/titanic
# https://www.kaggle.com/vbmokin/titanic-top-3-one-line-of-the-prediction-code
# https://www.kaggle.com/code/vbmokin/merging-fe-prediction-xgb-lgb-logr-linr
df = pd.concat([data,test_data],axis = 0).reset_index(drop = True)
# df = data.copy()
df['Title'] = df.Name.str.split(',').str[1].str.split('.').str[0].str.strip()
df['IsWomanOrBoy'] = ((df.Title == 'Master') | (df.Sex == 'female'))
df['LastName'] = df.Name.str.split(',').str[0]
family = df.groupby(df.LastName).Survived
df['WomanOrBoyCount'] = family.transform(lambda s: s[df.IsWomanOrBoy].fillna(0).count())
df['FamilySurvivedCount'] = family.transform(lambda s: s[df.IsWomanOrBoy].fillna(0).sum())
df['WomanOrBoySurvived'] = df.FamilySurvivedCount / df.WomanOrBoyCount
df.WomanOrBoyCount = df.WomanOrBoyCount
df['Alone'] = (df.WomanOrBoyCount == 0)

#Thanks to https://www.kaggle.com/kpacocha/top-6-titanic-machine-learning-from-disaster
#"Title" improvement
df['Title'] = df['Title'].replace('Ms','Miss')
df['Title'] = df['Title'].replace('Mlle','Miss')
df['Title'] = df['Title'].replace('Mme','Mrs')
# Embarked
df['Embarked'] = df['Embarked'].fillna('S')
# Cabin, Deck
df['Deck'] = df['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')
#df.loc[(df['Deck'] == 'T'), 'Deck'] = 'A'

# Thanks to https://www.kaggle.com/erinsweet/simpledetect
# Fare
med_fare = df.groupby(['Pclass', 'Parch', 'SibSp']).Fare.median()[3][0][0]
df['Fare'] = df['Fare'].fillna(med_fare)
df['Fare'] = pd.qcut(df['Fare'], 13)
#Age
df['Age'] = df.groupby(['Sex', 'Pclass', 'Title'])['Age'].apply(lambda x: x.fillna(x.median()))
df['Age'] = pd.qcut(df['Age'], 8)
# Family_Size
df['Family_Size'] = df['SibSp'] + df['Parch'] + 1

# Thanks to https://www.kaggle.com/vbmokin/titanic-top-3-cluster-analysis
cols_to_drop = ['Name','Ticket','Cabin', 'IsWomanOrBoy', 'WomanOrBoyCount', 'FamilySurvivedCount']
df = df.drop(cols_to_drop, axis=1)

df.WomanOrBoySurvived = df.WomanOrBoySurvived.fillna(0)
df.WomanOrBoySurvived = df.WomanOrBoySurvived.astype(int)
# df['WomanOrBoySurvived'] = pd.qcut(df['WomanOrBoySurvived'], 10)
df.Alone = df.Alone.fillna(0)

train_processed = df[df['Survived'].notnull()]
test_processed = df[df['Survived'].isnull()]
test_processed = test_processed.drop(['Survived'], axis=1)

## 결측값 확인

In [638]:
df['Fare']

0          (-0.001, 7.25]
1        (56.496, 83.475]
2           (7.896, 8.05]
3        (34.157, 56.496]
4           (7.896, 8.05]
              ...        
1304        (7.896, 8.05]
1305    (83.475, 512.329]
1306       (-0.001, 7.25]
1307        (7.896, 8.05]
1308      (15.742, 23.25]
Name: Fare, Length: 1309, dtype: category
Categories (13, interval[float64, right]): [(-0.001, 7.25] < (7.25, 7.75] < (7.75, 7.896] <
                                            (7.896, 8.05] ... (26.55, 34.157] <
                                            (34.157, 56.496] < (56.496, 83.475] < (83.475, 512.329]]

In [639]:
def display_missing(df):    
    for col in df.columns.tolist():          
        print(f'{col} column missing values: {df[col].isnull().sum()}')
    print('\n')

# 모델링 준비작업

---

# 모델링 작업

In [784]:
class My_func:

  # __init__
  def __init__(self):
    self.result = 0
    self._features = []
    self._target = ''

  # 특성 및 타겟 setter, getter 작성
  @property
  def features(self):
    return self._features
  
  @features.setter
  def features(self, data):
    values = data.drop(columns = self._target).columns
    self._features = list(values)

  @property
  def target(self):
    return self._target
  
  @target.setter
  def target(self, value):
    target = value
    self._target = target

  # 훈련/검증/테스트 데이터 나누는 함수
  def split_data(self, data):
    train, test = train_test_split(data, train_size=0.80, test_size=0.20, random_state=2)
    return train, test

  # 특성과 타겟값 분리하는 함수
  def split_target(self, data, features, target):
    X_data = data[features]
    y_data = data[target]

    return X_data, y_data

  # 인코딩하는 함수
  def encoding(self, X_train, X_test):
    ord = OrdinalEncoder()

    X_train_enc = ord.fit_transform(X_train)
    X_test_enc = ord.transform(X_test)

    return X_train_enc, X_test_enc

  # 생존자예측 XGBClassifier 모델 함수
  def modeling(self):
    xgb_model = XGBClassifier(
        n_jobs=-1,  
        random_state=1, 
        # criterion='entropy',
        # n_estimators=300,
        scale_pos_weight = 0.6/0.4 ,
        # gamma = 0.5
        )
    
    param_distributions = { 
      'n_estimators' : [100,200],
      'colsample_bylevel':[1,0.9,0.8],
      'colsample_bytree':[1,0.9,0.8],
      'learning_rate':[0.01,0.02,0.03],
      'max_depth': [5,6,7], 
      'min_samples_leaf': [10],
      "subsample":[0.7,0.8,0.9],
      "gamma":[0.5,1]
    }
    
    clf = GridSearchCV(
    xgb_model, 
    param_grid=param_distributions, 
    # n_iter=50, 
    cv=3, 
    # scoring='auc', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1, 
    # random_state=2
    )
    
    return clf

In [785]:
# 클래스 선언
fuc = My_func()
fuc.target = 'Survived'
fuc.features = df

# fuc.target, fuc.features

In [786]:
# 함수 적용
train, val = fuc.split_data(df) # 훈련/검증 데이터로 나누기
X_train, y_train = fuc.split_target(train, fuc.features, fuc.target) # 훈련데이터 특성과 타겟값을 분리
X_val, y_val = fuc.split_target(val, fuc.features, fuc.target) # 검증데이터 특성과 타겟값을 분리
X_train_enc, X_val_enc = fuc.encoding(X_train, X_val) # 인코딩 작업

# X_train_enc.shape, y_train.shape, X_val_enc.shape, y_val.shape

In [787]:
# 모델링
model = fuc.modeling()
model.fit(X_train_enc , y_train)

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(n_jobs=-1, random_state=1,
                                     scale_pos_weight=1.4999999999999998),
             n_jobs=-1,
             param_grid={'colsample_bylevel': [1, 0.9, 0.8],
                         'colsample_bytree': [1, 0.9, 0.8], 'gamma': [0.5, 1],
                         'learning_rate': [0.01, 0.02, 0.03],
                         'max_depth': [5, 6, 7], 'min_samples_leaf': [10],
                         'n_estimators': [100, 200],
                         'subsample': [0.7, 0.8, 0.9]},
             return_train_score=True, verbose=10)

In [788]:
y_pred = model.predict(X_val_enc)

In [789]:
acc = accuracy_score(y_pred, y_val)
print('최적 하이퍼파라미터: ', model.best_params_)
print(f'정확도 : {acc}')

최적 하이퍼파라미터:  {'colsample_bylevel': 0.8, 'colsample_bytree': 0.9, 'gamma': 1, 'learning_rate': 0.03, 'max_depth': 5, 'min_samples_leaf': 10, 'n_estimators': 100, 'subsample': 0.9}
정확도 : 0.8379888268156425


# 테스트셋으로 예측한후 내보내기

In [827]:
# X_test = eda(test)
X_test = test_processed

In [828]:
# 클래스 선언
fuc_test = My_func()
fuc_test.target = 'Survived'
fuc_test.features = df

# fuc.target, fuc.features

In [829]:
# 함수 적용
X_train, y_train = fuc_test.split_target(df, fuc_test.features, fuc_test.target) # 훈련데이터 특성과 타겟값을 분리
X_train_enc, X_test_enc = fuc_test.encoding(X_train, X_test) # 인코딩 작업

In [830]:
# # 모델링
# model = fuc.modeling()
# model.fit(X_train_enc, y_train)

In [831]:
y_pred_test = model.predict(X_test_enc)
submission['Survived'] = y_pred_test

# 파일뒤에 현재시간 기록하기

In [778]:
# # Get the current time in the UTC+9 timezone
# now_utc_9 = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

# # Format the datetime object as a string
# utc_9_time_str = now_utc_9.strftime('%Y-%m-%d-%H-%M-%S')

# 모델 피클링해서 내보내기

In [832]:
import pickle

# Get the current time in the UTC+9 timezone
now_utc_9 = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

# Format the datetime object as a string
utc_9_time_str = now_utc_9.strftime('%Y-%m-%d-%H-%M-%S')

model_name = f'model_{utc_9_time_str}.pkl'
with open(model_name,'wb') as pickle_file:
    pickle.dump(model, pickle_file)

files.download(model_name )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# submission파일 내컴퓨터로 내보내기

In [833]:
from google.colab import files

# Get the current time in the UTC+9 timezone
now_utc_9 = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

# Format the datetime object as a string
utc_9_time_str = now_utc_9.strftime('%Y-%m-%d-%H-%M-%S')

file_name = f"titanic_submission_{utc_9_time_str}.csv"
submission.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>